In [59]:
import pandas as pd
import glob,os

import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
plotly.tools.set_config_file(world_readable=True)

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics as smg
import statsmodels.stats as sms

# Getting Data

In [3]:
sourcepath = 'S:/Chris/Natural Rates of Severity'
os.chdir(sourcepath)
glob.glob('*.dta')

['CC growth rates stats sheet.dta',
 'Demography 2003-2008 crystal creek.dta',
 'Growth Rates and some regression analyses (goodness of fit).dta',
 'Growth Rates and some regression analyses.dta',
 'Middendorf and Agard data includign sv and sj.dta',
 'Middendorf and Agard data sv for growth.dta',
 'Middendorf and Agard growth rates stats sheet.dta',
 'Middendorf and Agard natural rates & severity of autotomy.dta',
 'Natural Severity of TL model.dta',
 'Regression without juv only intact.dta',
 'regrown.dta',
 'RTL.dta',
 'tlsvl ratio intact.dta',
 'TL_SVL on growth (long).dta',
 'TL_SVL on growth (long)2.dta',
 'TL_SVL on growth 3.dta',
 'TL_SVL on growth.dta']

## Read in and Describe the Data

In [4]:
df = pd.read_stata('Natural Severity of TL model.dta')
print(df.describe(),'\n\n',['{}: {}'.format(x,df[x].unique()) for x in df.columns if df[x].nunique()<10])
df

              year        svl          tl   rtl       mass
count    89.000000  89.000000   89.000000  89.0  64.000000
mean   2008.808989  71.910112   98.011236   0.0  12.531250
std       1.851758   8.661094   12.144485   0.0   4.821928
min    2005.000000  54.000000   74.000000   0.0   5.000000
25%    2007.000000  65.000000   90.000000   0.0   8.500000
50%    2009.000000  71.000000   97.000000   0.0  11.500000
75%    2010.000000  79.000000  107.000000   0.0  16.000000
max    2011.000000  90.000000  126.000000   0.0  25.000000 

 ['year: [2005 2006 2011 2010 2007 2009]', 'species: [S. jarrovii]\nCategories (1, object): [S. jarrovii]', 'sex: [female, male]\nCategories (2, object): [female < male]', 'rtl: [0]']


,year,lizard,species,sex,svl,tl,rtl,mass,toes
0,2005,p9c,S. jarrovii,female,84,100,0,19.50,18
1,2005,p30c,S. jarrovii,female,69,102,0,11.50,1-6-13-20
2,2005,p31c,S. jarrovii,female,70,92,0,11.00,1-6-15-16
3,2005,p14c,S. jarrovii,female,84,113,0,17.00,5-7
4,2006,w8R,S. jarrovii,female,64,98,0,8.00,`15
5,2006,w10K,S. jarrovii,female,79,112,0,13.50,`17
6,2011,y2b,S. jarrovii,female,75,98,0,10.50,1-10-11-16
7,2010,w2a,S. jarrovii,female,66,94,0,7.50,1-10-11-18
8,2010,w18a.t,S. jarrovii,female,65,90,0,8.50,1-10-12-19
9,2010,w26a,S. jarrovii,female,70,97,0,9.00,1-10-13-18


In [53]:
df2 = pd.read_stata('Middendorf and Agard growth rates stats sheet.dta')
print(df2.describe(),'\n\n',['{}: {}'.format(x,df2[x].unique()) for x in df2.columns if df2[x].nunique()<10])
df2

           lizard        year1       svl1         tl1       rtl1  \
count   60.000000    60.000000  60.000000   60.000000  60.000000   
mean   188.983333  2003.983333  60.350000   74.166667   5.666667   
std     96.947040     1.081300  18.199297   25.403301  11.664326   
min      2.000000  2003.000000  29.000000   12.000000   0.000000   
25%    113.000000  2003.000000  37.750000   50.750000   0.000000   
50%    209.000000  2004.000000  67.000000   83.000000   0.000000   
75%    255.250000  2005.000000  71.250000   90.500000   2.250000   
max    387.000000  2008.000000  91.000000  126.000000  50.000000   

       tailcondition1      mass1     meters1      tlnew1        year2  \
count       60.000000  60.000000   57.000000   60.000000    60.000000   
mean         1.566667   9.435833  196.508772   68.500000  2005.133333   
std          0.889995   6.837107  158.728106   27.795561     1.371213   
min          1.000000   0.500000  -40.000000   12.000000  2004.000000   
25%          1.000000 

,lizard,year1,svl1,tl1,rtl1,tailcondition1,mass1,meters1,tlnew1,year2,...,tln3,tln2,tln32,svl21ys,tl21ys,ntl21ys,rtl21ys,svl21ysT,tl21ysT,ntl21ysT
0,220,2003,68,76,25,3,11.400000,362.0,51.0,2004,...,49.0,31.0,18.0,10.000000,-45.000000,-20.000000,-25.000000,3.162278,NaN,NaN
1,320,2003,87,88,4,3,24.200001,-5.0,84.0,2004,...,85.0,89.0,-4.0,1.000000,1.000000,5.000000,-4.000000,1.000000,1.000000,2.236068
2,138,2003,30,40,0,1,0.900000,3.0,40.0,2004,...,38.0,86.0,-48.0,30.000000,46.000000,46.000000,0.000000,NaN,NaN,NaN
3,240,2004,30,36,0,1,0.700000,19.0,36.0,2005,...,NaN,83.0,NaN,31.000000,47.000000,47.000000,0.000000,NaN,NaN,NaN
4,308,2003,65,90,0,1,9.500000,355.0,90.0,2004,...,48.0,96.0,-48.0,8.000000,6.000000,6.000000,0.000000,2.828427,2.449490,2.449490
5,11,2003,60,84,0,1,8.000000,413.0,84.0,2004,...,NaN,112.0,NaN,20.000000,28.000000,28.000000,0.000000,4.472136,5.291502,5.291502
6,212,2004,66,99,0,1,9.000000,-7.0,99.0,2005,...,99.0,97.0,2.0,5.000000,-2.000000,-2.000000,0.000000,2.236068,NaN,NaN
7,252,2004,62,89,0,1,7.800000,262.0,89.0,2005,...,NaN,98.0,NaN,8.000000,9.000000,9.000000,0.000000,2.828427,3.000000,3.000000
8,114,2003,65,99,0,1,13.500000,392.0,99.0,2005,...,NaN,120.0,NaN,11.500000,10.500000,10.500000,0.000000,3.391165,3.240370,3.240370
9,284,2005,74,104,0,1,13.500000,385.0,104.0,2006,...,NaN,113.0,NaN,8.000000,9.000000,9.000000,0.000000,2.828427,3.000000,3.000000


# Visualization
Here we visualize the relationship between SVL and TL.

[Back to Table of Contents](#Table-of-Contents)

## SVL and TL

In [36]:
Males = go.Scatter(x=df.loc[df.sex=='female','svl'],y=df.loc[df.sex=='female','tl'],name='Female',mode = 'markers')
Females = go.Scatter(x=df.loc[df.sex=='male','svl'],y=df.loc[df.sex=='male','tl'],name='Male',mode = 'markers')
data =[Males,Females]
layout = go.Layout(
    title = 'Relationship Between SVL and TL in Intact <i>Sceloporus jarrovii</i>',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 5,
        title = 'SVL (mm)',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18),
        dtick = 10,
    range = (0,df.tl.max()+5)
))
fig = go.Figure(data=data, layout=layout)
plot(fig, filename = 'Relationship Between SVL and TL in Intact Sceloporus jarrovii.html')
iplot(fig, filename = 'Relationship Between SVL and TL in Intact Sceloporus jarrovii.html')

In [23]:
SVLMales = go.Histogram(x=df.loc[df.sex=='female','svl'],name='Male SVL')
SVLFemales = go.Histogram(x=df.loc[df.sex=='female','svl'],name='Female SVL')
data =[SVLMales,SVLFemales]
layout = go.Layout(
    title = 'Distribution of SVL in Intact <i>Sceloporus jarrovii</i> by Sex',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 5,
        title = 'Length(mm)',
        titlefont = dict(
            size = 18)))
fig = go.Figure(data=data, layout=layout)
plot(fig, filename = 'Distribution of SVL in Intact Sceloporus jarrovii by Sex.html')
iplot(fig, filename = 'Distribution of SVL in Intact Sceloporus jarrovii by Sex.html')

In [25]:
TLMales = go.Histogram(x=df.loc[df.sex=='female','tl'],name='Male TL')
TLFemales = go.Histogram(x=df.loc[df.sex=='female','tl'],name='Female TL')
data =[TLMales,TLFemales]
layout = go.Layout(
    title = 'Distribution of TL in Intact <i>Sceloporus jarrovii</i> by Sex',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 5,
        title = 'Length(mm)',
        titlefont = dict(
            size = 18)))
fig = go.Figure(data=data, layout=layout)
plot(fig, filename = 'Distribution of TL in Intact Sceloporus jarrovii by Sex.html')
iplot(fig, filename = 'Distribution of TL in Intact Sceloporus jarrovii by Sex.html')

## Growth

### SVL

In [64]:
df2.loc[df2.tailcondition1==1,'tailcondition'] = 'intact'
df2.loc[df2.tailcondition1!=1,'tailcondition'] = 'autotomized'

In [58]:
SVLMales = go.Histogram(x=df2.loc[df2.sex==1,'svl21ys'],name='Male SVL Growth')
SVLFemales = go.Histogram(x=df2.loc[df2.sex==0,'svl21ys'],name='Female SVL Growth')
data =[SVLMales,SVLFemales]
layout = go.Layout(
    title = 'Distribution of SVL Growth in Intact <i>Sceloporus jarrovii</i> by Sex',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'SVL Growth(mm)',
        titlefont = dict(
            size = 18)))
fig = go.Figure(data=data, layout=layout)
plot(fig, filename = 'Distribution of SVL Growth in Intact Sceloporus jarrovii by Sex.html')
iplot(fig, filename = 'Distribution of SVL Growth in Intact Sceloporus jarrovii by Sex.html')

These data are not normally distributed, so we will use a non-parametric test (Wilcoxon).

# Check for difference in Growth Rates

In [66]:
import scipy.stats as stats

## SVL

Here we check to see if SVL growth rate differs between Intact and Autotomized SVL. We will exclude juveniles for this analysis since we expect juveniles to grow faster and there are no juveniles in the autotomized group.

In [88]:
print(stats.kruskal(df2.loc[(df2.svl1>=54)&(df2.tailcondition=='intact')].svl21ys,
                    df2.loc[(df2.svl1>=54)&(df2.tailcondition=='autotomized')].svl21ys))
print("Intact SVL: {}\nAutotomized SVL: {}"\
      .format(df2.loc[(df2.svl1>=54)&(df2.tailcondition=='intact')].svl21ys.describe(),
              df2.loc[(df2.svl1>=54)&(df2.tailcondition=='autotomized')].svl21ys.describe()))

KruskalResult(statistic=1.1170006912763095, pvalue=0.29056520049479184)
Intact SVL: count    26.000000
mean      6.980769
std       3.986178
min       0.000000
25%       5.000000
50%       7.000000
75%       8.000000
max      20.000000
Name: svl21ys, dtype: float64
Autotomized SVL: count    16.000000
mean      5.416667
std       3.907258
min       0.000000
25%       2.000000
50%       5.000000
75%       9.250000
max      11.000000
Name: svl21ys, dtype: float64


SVL growth rates are equal with juveniles excluded.

## TL

Here we check to see if tl growth rate differs between Intact and Autotomized tl. We will exclude juveniles for this analysis since we expect juveniles to grow faster and there are no juveniles in the autotomized group.

In [92]:
print(stats.kruskal(df2.loc[(df2.tl1>=54)&(df2.tailcondition=='intact')].tl21ys,
                    df2.loc[(df2.tl1>=54)&(df2.tailcondition=='autotomized')].tl21ys))
print("Intact tl: {}\nAutotomized tl: {}"\
      .format(df2.loc[(df2.tl1>=54)&(df2.tailcondition=='intact')].tl21ys.describe(),
              df2.loc[(df2.tl1>=54)&(df2.tailcondition=='autotomized')].tl21ys.describe()))

KruskalResult(statistic=0.017321974463654147, pvalue=0.8952904244188784)
Intact tl: count    27.000000
mean      3.851852
std      12.603277
min     -32.000000
25%      -3.000000
50%       6.000000
75%      10.000000
max      28.000000
Name: tl21ys, dtype: float64
Autotomized tl: count    15.000000
mean      2.355556
std      13.695364
min     -45.000000
25%       1.500000
50%       5.000000
75%      10.000000
max      12.000000
Name: tl21ys, dtype: float64


The negative tl growth values for intact tail individuals suggest a need for cleaning data.  

## Regression

## Fit Model

In [90]:
results = smf.ols('df.tl~df.svl+df.sex',data = df.loc[(df.svl>=54)]).fit()

## Inspect Results

In [91]:
resultsfile = open('results.txt','w')
summary = str(results.summary())
# summary
resultsfile.write(summary)
resultsfile.close() 
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  df.tl   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     240.7
Date:                Sun, 21 Jul 2019   Prob (F-statistic):           5.84e-36
Time:                        16:12:17   Log-Likelihood:                -264.05
No. Observations:                  89   AIC:                             534.1
Df Residuals:                      86   BIC:                             541.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         10.9179      4.674      2.336      0.022       1.625      20.210
df.sex[T.male]     3.2556      1.224      2.659      0.009       0.822       5.689
df.svl             1.1959      0.067     17.798      0.000       1.062       1.329
==============================================================================
Omnibus:                        1.912   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.384   Jarque-Bera (JB):                1.347
Skew:                          -0.125   Prob(JB):                        0.510
Kurtosis:                       3.548   Cond. No.                         672.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""